<a href="https://colab.research.google.com/github/taavip/PLANK/blob/main/plank_statistika_kvaliteedi_analyys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install owslib geopandas pandas

In [ ]:
import geopandas as gpd
import requests
from owslib.wfs import WebFeatureService
from concurrent.futures import ThreadPoolExecutor, as_completed
import time  # Aja mõõtmise jaoks
import math

# WFS teenuse URL
url = 'https://planeeringud.ee/plank/wfs?'

# Loome ühenduse WFS teenusega
wfs = WebFeatureService(url=url, version="2.0.0")

# Loeme kõik saadaval olevad kihid teenuses
layers = list(wfs.contents)

# Kuvame kõik saadaval olevad kihid
print("Saadaval kihid WFS teenuses:")
for layer in layers:
    print(layer)

# Funktsioon, mis laadib kihi andmed WFS teenusest
def load_layer_data(layer_name):
    start_time = time.time()  # Alustame aja mõõtmist
    print(f"\nLaen kihti: {layer_name}")  # Kuvame, millist kihti hetkel laeme

    # Määrame päringu parameetrid
    params = dict(service='WFS', version="2.0.0", request='GetFeature',
                  typeNames=layer_name, startIndex=0, count=1000, outputFormat='text/xml; subtype=gml/3.2.1')

    all_gdf = []
    page = 0

    while True:
        # Arvutame startIndex järgmisele lehele
        params['startIndex'] = page * 1000

        # Koostame päringu URL
        wfs_request_url = requests.Request('GET', url, params=params).prepare().url
        response = requests.get(wfs_request_url)

        # Laeme andmed GeoPandasiga
        try:
            gdf = gpd.read_file(wfs_request_url)
            all_gdf.append(gdf)

            # Kui tagastati vähem kui 1000 kirjet, lõpetame
            if len(gdf) < 1000:
                break

            # Jätkame järgmise lehega
            page += 1

        except Exception as e:
            print(f"Viga kihi {layer_name} laadimisel lehelt {page + 1}: {e}")
            break

    # Ühendame kõik lehed üheks GeoDataFrame'iks
    if all_gdf:
        combined_gdf = gpd.GeoDataFrame(pd.concat(all_gdf, ignore_index=True))
        end_time = time.time()  # Lõpetame aja mõõtmise
        load_time = end_time - start_time
        print(f"Kihi {layer_name} laadimine kestis {load_time:.2f} sekundit.")
        return combined_gdf
    else:
        print(f"Ei leitud andmeid kihil {layer_name}.")
        return None

# Lae kõik kihid paralleelselt
def load_all_layers_parallel(layers):
    start_time = time.time()  # Alustame kogu protsessi ajamõõtmist
    gdfs = {}
    with ThreadPoolExecutor(max_workers=8) as executor:  # Tõstetud töötlejate arv
        # Laadime kõik kihid paralleelselt ja kogume tulemusi
        futures = {executor.submit(load_layer_data, layer): layer for layer in layers}

        for future in as_completed(futures):
            layer = futures[future]
            try:
                gdf = future.result()
                if gdf is not None:
                    gdfs[layer] = gdf
                    print(f"Kihil {layer} on {len(gdf)} rida andmeid.")
            except Exception as e:
                print(f"Kihi {layer} laadimisel tekkis viga: {e}")

    end_time = time.time()  # Lõpetame kogu protsessi ajamõõtmise
    total_time = end_time - start_time
    print(f"Kõigi kihtide laadimine kestis kokku {total_time:.2f} sekundit.")

    return gdfs

# Laeme kõik kihid paralleelselt (nt võtame ainult esimese 10 kihi)
gdfs = load_all_layers_parallel(layers)  # Eemaldage [20:50], kui tahate kõiki kihte laadida

# Kontrollime tulemusi
print(f"Leitud GeoDataFrame'id: {len(gdfs)}")
for layer_name, gdf in gdfs.items():
    print(f"Kihi {layer_name} veerud: {gdf.columns}")


Saadaval kihid WFS teenuses:
plank:yrpmsp
plank:yrpmsp_kihid
plank:yrpmsp_kihid_2
plank:yrpmsp_kihid_1
plank:maakonnaplaneeringu_ala
plank:mp_msp
plank:mp_msp_2
plank:mp_msp_1
plank:mp_tuuleala
plank:mp_vesiviljelus
plank:mp_rohev
plank:mp_toimepiirkond
plank:mp_linnaline
plank:mp_suunis
plank:mp_suunis_2
plank:mp_suunis_1
plank:mp_vaartala
plank:mp_vaartala_2
plank:mp_vaartala_1
plank:mp_taristu
plank:mp_taristu_2
plank:mp_taristu_1
plank:mp_keskus
plank:yldplaneeringud
plank:yp_tiheasustus
plank:yp_maakasutus
plank:yp_dpkoko
plank:yp_ekv
plank:yp_ekv_1
plank:yp_tingimus
plank:yp_tingimus_2
plank:yp_tingimus_1
plank:yp_ehitis
plank:yp_ehitis_2
plank:yp_ehitis_1
plank:yp_transport_3
plank:yp_transport
plank:yp_transport_1
plank:detailplaneering
plank:dp_krunt
plank:dp_hoonestus
plank:dp_tingimus
plank:dp_tingimus_2
plank:dp_tingimus_1
plank:dp_transp_3
plank:dp_transp
plank:dp_transp_1
plank:dp_tehno_3
plank:dp_tehno
plank:dp_tehno_1
plank:kep_ala
plank:kep_krunt
plank:kep_hoonestus
pl

In [ ]:
gdfs['plank:detailplaneering']

,gml_id,lowerCorner,upperCorner,sysid,planid,kovid,plannim,korraldaja,planseis_nimi,planksh,muutev,planeesm,timePosition,vastuvkp|timePosition,kehtestkp|timePosition,url,failid,planviide,tyhistkp|timePosition,geometry
0,detailplaneering.30103271,6604090.780000 592842.450000,6604572.430000 593241.110000,30103271,5050.0,15,Hara kÃ¼la Kadapiku kinnistu detailplaneering,Kuusalu Vallavalitsus,kehtiv,ei,ei,"katastriÃ¼ksuse kruntideks jagamine, sihtotsta...",2002-04-18,2002-10-15,2003-01-23,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((593241.11 6604549.71, 593210.85 6604..."
1,detailplaneering.30101799,6405682.170000 621397.610000,6405747.660000 621485.970000,30101799,NaN,DP2001-002,"Valga linna kvartalis nr 15 Kungala 1a, 3, 5, ...",Valga Vallavalitsus,kehtiv,ei,ei,Kruntide piiride määramine,2001-03-07,2001-06-20,2001-08-30,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((621451.05 6405738.29, 621468.99 6405..."
2,detailplaneering.30100972,6595534.940000 582521.440000,6595722.100000 582744.540000,30100972,326.0,20,Männisalu 1 ja 2,Kuusalu Vallavalitsus,kehtiv,ei,ei,jagamine elamukruntideks,2002-06-26,None,2003-01-30,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((582737.99 6595654.54, 582696.59 6595..."
3,detailplaneering.30102577,6582856.460000 556888.110000,6583082.480000 557210.770000,30102577,151.0,1/05,"Põllu ja Vilja, Kulli",Raasiku Vallavalitsus,kehtiv,ei,ei,elamukruntideks jagamine,2005-01-11,2006-08-10,2006-10-10,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((556907.88 6582930.85, 556897.92 6582..."
4,detailplaneering.30100074,6591297.990000 578582.900000,6591408.350000 578731.270000,30100074,NaN,None,Kiiu aleviku Vana-Narva mnt 11 kinnistu detail...,Kuusalu Vallavalitsus,kehtiv,ei,ei,Vana-Narva mnt 11 kinnistule ehitusõiguse sead...,2017-10-27,2022-05-19,2022-12-15,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((578632.22 6591297.99, 578582.9 65914..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806,detailplaneering.30100842,6576904.057053 722126.295227,6577197.342927 722587.982266,30100842,31127.0,851000088,Auvere külas asuv korstnate maa-ala detailplan...,Narva-Jõesuu Linnavalitsus,kehtetu,ei,ei,Ehitusõiguse andmine Eesti elektrijaama plokki...,2013-06-26,2013-11-12,2013-12-10,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,2019-08-28,"POLYGON ((722126.295 6576966.099, 722568.792 6..."
2807,detailplaneering.30103305,6583575.531900 666928.339500,6584167.987300 667560.881486,30103305,16642.0,None,Kiviõli Ettevõtlusala I territooriumi detailpl...,Lüganuse Vallavalitsus,kehtetu,ei,ei,Detailplaneeringu koostamise eesmärgiks on pla...,2009-11-10,2011-04-13,2022-02-17,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,https://drive.google.com/drive/folders/1ZuzxS_...,2022-02-17,"POLYGON ((667317.686 6584147.723, 667325.373 6..."
2808,detailplaneering.30103312,6582977.552122 666428.262351,6583608.525094 667012.554871,30103312,3492.0,None,Kiviõli Ettevõtlusala II territooriumi detailp...,Lüganuse Vallavalitsus,kehtetu,ei,jah,Luua vastavalt planerimisseadusele alusdokumen...,2007-11-29,2009-08-12,2009-12-17,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,https://drive.google.com/drive/folders/19mrtfs...,2022-02-17,"POLYGON ((666655.502 6583450.117, 667007.075 6..."
2809,detailplaneering.30100083,6415016.320000 679699.200000,6415341.720000 679992.120000,30100083,114292.0,232-DP,"Pikk tn 17, 17c, 17d ja 17e kinnistute detailp...",Võru Linnavalitsus,kehtetu,ei,ei,Olemasoleva to